In [151]:
import tensorflow as tf
import pickle
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras import layers, models


In [152]:
print(f"Tensor flow version: {tf.__version__}")

Tensor flow version: 2.10.0


## 1. Preparación de los datos

### Descargar el conjunto de datos de rótulos de tráfico que contiene las imágenes de las 43 clases mencionadas

In [153]:
# Cargar el archivo .p
with open('data/Datos_Rotulos_Trafico/entrenamiento.p', 'rb') as file:
    data = pickle.load(file)

# Imprimir las claves disponibles
print(data.keys())

# Inspeccionar un poco más el contenido
for key in data.keys():
    print(f"Key: {key}, Type: {type(data[key])}, Length: {len(data[key])}")


dict_keys(['coords', 'labels', 'features', 'sizes'])
Key: coords, Type: <class 'numpy.ndarray'>, Length: 34799
Key: labels, Type: <class 'numpy.ndarray'>, Length: 34799
Key: features, Type: <class 'numpy.ndarray'>, Length: 34799
Key: sizes, Type: <class 'numpy.ndarray'>, Length: 34799


### Dividir el conjunto de datos en conjuntos de entrenamiento, validación y prueba.

In [154]:
# Función para cargar los datos desde un archivo .p
def load_data(pickle_file):
    with open(pickle_file, 'rb') as file:
        data = pickle.load(file)
    return data['features'], data['labels']

In [155]:
# Cargar los conjuntos de datos
X_train, y_train = load_data('data/Datos_Rotulos_Trafico/entrenamiento.p')
X_val, y_val = load_data('data/Datos_Rotulos_Trafico/validacion.p')
X_test, y_test = load_data('data/Datos_Rotulos_Trafico/prueba.p')


### Realizar preprocesamiento de las imágenes, como redimensionarlas a un tamaño estándar, normalización, etc

In [156]:
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Convertir etiquetas a arrays de NumPy (aunque ya están en ese formato, es bueno asegurarse)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

X_train = tf.image.rgb_to_grayscale(X_train)
X_val = tf.image.rgb_to_grayscale(X_val)
X_test = tf.image.rgb_to_grayscale(X_test)

In [157]:
# Verificar la forma de los datos
print(f'Train data shape: {X_train.shape}, Labels shape: {y_train.shape}')
print(f'Validation data shape: {X_val.shape}, Labels shape: {y_val.shape}')
print(f'Test data shape: {X_test.shape}, Labels shape: {y_test.shape}')

Train data shape: (34799, 32, 32, 1), Labels shape: (34799,)
Validation data shape: (4410, 32, 32, 1), Labels shape: (4410,)
Test data shape: (12630, 32, 32, 1), Labels shape: (12630,)


## 2. Implementación de la arquitectura Le-Net:

### Presentar la arquitectura Le-Net en detalle, explicando cada capa (convolucional, pooling, fully connected).

LeNet es una de las primeras arquitecturas de redes neuronales convolucionales, desarrollada por Yann LeCun y su equipo en 1998. Fue diseñada inicialmente para reconocer dígitos escritos a mano en imágenes de 28x28 píxeles, como los dígitos del conjunto de datos MNIST. A continuación, se detalla la arquitectura de LeNet:

#### 1. Capa Convolucional 1 (Conv1)
- **Entrada**: Una imagen de 32x32 píxeles (en blanco y negro, con un solo canal).
- **Operación**: Se aplica un filtro convolucional de tamaño 5x5, con 6 filtros en total, produciendo 6 mapas de características.
- **Salida**: 6 mapas de características, cada uno de tamaño 28x28 píxeles.

#### 2. Capa de Submuestreo (Pooling) 1 (Pool1)
- **Entrada**: Los 6 mapas de características de la capa anterior.
- **Operación**: Se aplica un max-pooling (o submuestreo) con un tamaño de ventana 2x2 y un stride de 2, lo que reduce las dimensiones de cada mapa de características.
- **Salida**: 6 mapas de características de tamaño 14x14 píxeles.

#### 3. Capa Convolucional 2 (Conv2)
- **Entrada**: Los 6 mapas de características de la capa anterior.
- **Operación**: Se aplican 16 filtros convolucionales de tamaño 5x5, produciendo 16 nuevos mapas de características.
- **Salida**: 16 mapas de características, cada uno de tamaño 10x10 píxeles.

#### 4. Capa de Submuestreo (Pooling) 2 (Pool2)
- **Entrada**: Los 16 mapas de características de la capa anterior.
- **Operación**: Se aplica max-pooling con una ventana de 2x2 y un stride de 2.
- **Salida**: 16 mapas de características de tamaño 5x5 píxeles.

#### 5. Capa Fully Connected (FC1)
- **Entrada**: Las salidas aplanadas de los 16 mapas de características de tamaño 5x5, lo que da un vector de 400 elementos (16x5x5).
- **Operación**: Se conecta cada una de las 400 entradas a 120 neuronas completamente conectadas.
- **Salida**: Un vector de 120 elementos.

#### 6. Capa Fully Connected (FC2)
- **Entrada**: El vector de 120 elementos de la capa anterior.
- **Operación**: Se conecta cada uno de los 120 valores a 84 neuronas completamente conectadas.
- **Salida**: Un vector de 84 elementos.

#### 7. Capa de Salida (Output Layer)
- **Entrada**: El vector de 84 elementos.
- **Operación**: Se conecta a una capa de salida con tantas neuronas como clases de salida, en este caso 10 (para los dígitos del 0 al 9 en MNIST).
- **Salida**: Un vector de 43 elementos

En resumen, la arquitectura LeNet es un ejemplo clásico de una red neuronal convolucional, que combina capas convolucionales para extracción de características y capas completamente conectadas para la clasificación. Aunque sencilla en comparación con arquitecturas modernas, LeNet sentó las bases para muchos avances en el campo de la visión por computadora.


### Mostrar el diseño de la red Le-Net utilizando una herramienta de diagramación.

![Diagrama en blanco (1).png](<attachment:Diagrama en blanco (1).png>)

La red LeNet toma como entrada una imagen de 32x32 píxeles y la procesa a través de varias capas para extraer características y clasificarlas. El flujo de la red es el siguiente:

1. **Input Layer**: Recibe la imagen de entrada de 32x32 píxeles con un solo canal (en blanco y negro).

2. **Conv1 Layer**: Aplica 6 filtros convolucionales de tamaño 5x5 para generar 6 mapas de características, cada uno de 28x28 píxeles.

3. **Pool1 Layer**: Realiza una operación de submuestreo (pooling) para reducir el tamaño de los mapas de características a 14x14 píxeles.

4. **Conv2 Layer**: Aplica 16 filtros convolucionales de tamaño 5x5, produciendo 16 mapas de características de 10x10 píxeles.

5. **Pool2 Layer**: Aplica otro submuestreo para reducir el tamaño de los mapas de características a 5x5 píxeles.

6. **FC1 Layer**: Aplana las características y las pasa a través de una capa completamente conectada con 120 neuronas.

7. **FC2 Layer**: Las salidas de la capa anterior se pasan a otra capa completamente conectada con 84 neuronas.

8. **Output Layer**: Finalmente, la capa de salida consiste en 10 neuronas, que corresponden a las 10 clases posibles para la clasificación.

Este flujo de procesamiento permite que la red LeNet extraiga y combine características a diferentes niveles de abstracción, culminando en la predicción de la clase de la imagen de entrada.


### Explicar el proceso de convolución, función de activación y pooling.

**Proceso de Convolución**

El proceso de convolución es una operación fundamental en las redes neuronales convolucionales (CNNs) que consiste en aplicar un filtro (o kernel) sobre la entrada (como una imagen) para extraer características importantes. El filtro se desliza (o convoluciona) sobre la entrada, realizando multiplicaciones punto a punto entre los valores del filtro y las secciones correspondientes de la imagen. Luego, se suman los productos resultantes para obtener un solo valor que representa la activación en esa ubicación específica. Este proceso se repite para cada posición del filtro, produciendo un mapa de características que resalta las características detectadas por el filtro en la imagen original.

**Función de Activación**

Una vez que la convolución se ha aplicado, el resultado suele pasar a través de una función de activación, siendo la más común la función ReLU (Rectified Linear Unit). La función ReLU se define como \( f(x) = \max(0, x) \), es decir, reemplaza todos los valores negativos por cero y deja los valores positivos sin cambios. Esta no linealidad permite a la red aprender una amplia gama de patrones complejos y aumentar su capacidad de representación. La función de activación se aplica a cada valor en el mapa de características generado por la convolución.

**Pooling**

El pooling es una operación de reducción de dimensionalidad que se aplica sobre los mapas de características después de la convolución y la activación. La operación más común es el Max Pooling, donde se selecciona el valor máximo dentro de una ventana (por ejemplo, de 2x2) que se desliza sobre el mapa de características. El pooling reduce el tamaño del mapa de características, haciendo que la red sea más computacionalmente eficiente y menos propensa a sobreajustarse. También ayuda a que la red sea invariante a pequeñas traslaciones de la entrada, mejorando la capacidad de generalización del modelo.


## 3. Construcción del modelo:

### Utilizar la biblioteca de aprendizaje profundo TensorFlow para construir la arquitectura Le-Net

In [158]:
model = models.Sequential()

### Definir la estructura de capas convolucionales, capas de pooling y capas fully connected.

In [159]:
# Capa de convolución 1
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.AveragePooling2D())

# Capa de convolución 2
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model.add(layers.AveragePooling2D())

# Aplanar las salidas
model.add(layers.Flatten())

# Capa completamente conectada 1
model.add(layers.Dense(units=120, activation='relu'))

# Capa completamente conectada 2
model.add(layers.Dense(units=84, activation='relu'))

# Capa de salida
model.add(layers.Dense(units=43, activation='softmax'))




In [160]:
# Resumen del modelo
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_6 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_28 (Conv2D)          (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_7 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten_12 (Flatten)        (None, 400)               0         
                                                                 
 dense_32 (Dense)            (None, 120)             

### Explicar la importancia de la función de pérdida y el optimizador.

**Función de Pérdida**

La función de pérdida es un componente crucial en el entrenamiento de una red neuronal. Su principal objetivo es cuantificar cuán lejos están las predicciones del modelo de las verdaderas etiquetas. En otras palabras, mide el "costo" o "error" asociado a las predicciones actuales de la red. Durante el entrenamiento, el modelo intenta minimizar esta función de pérdida, ajustando los pesos de la red para que las predicciones se acerquen lo más posible a las etiquetas correctas. Existen diferentes tipos de funciones de pérdida, como la entropía cruzada para tareas de clasificación o el error cuadrático medio para tareas de regresión, cada una adecuada para diferentes tipos de problemas.

**Optimizador**

El optimizador es el algoritmo que ajusta los pesos y sesgos de la red neuronal en función del valor de la función de pérdida. Su papel es minimizar la función de pérdida mediante un proceso iterativo de ajuste de los parámetros del modelo. Uno de los optimizadores más comunes es el Descenso de Gradiente Estocástico (SGD), que ajusta los parámetros en la dirección opuesta al gradiente de la función de pérdida con respecto a los parámetros. Variantes más avanzadas como Adam, RMSprop, y AdaGrad, entre otros, incluyen mejoras para manejar la velocidad de aprendizaje de manera adaptativa y reducir la variabilidad del descenso de gradiente, acelerando así la convergencia y mejorando la estabilidad del entrenamiento.

**Importancia Conjunta**

La combinación de una función de pérdida y un optimizador es fundamental para el entrenamiento efectivo de una red neuronal. Mientras que la función de pérdida proporciona una medida del error que el modelo debe minimizar, el optimizador determina cómo el modelo debe actualizar sus pesos para reducir ese error. Juntos, estos componentes permiten que la red aprenda patrones complejos en los datos, mejorando su capacidad para hacer predicciones precisas.


# Entrenamiento del modelo

El proceso de entrenamiento de la red neuronal comienza con la carga y normalización de los datos, escalando las imágenes al rango de [0, 1] para mejorar la eficiencia. La red, se basa en la arquitectura LeNet, incluye capas convolucionales y de pooling para extraer características y reducir la dimensionalidad, seguidas por capas densas que realizan la clasificación final. Durante el entrenamiento, se utiliza un optimizador para ajustar los pesos de la red mediante mediante la minimización de una función de pérdida, mediante retropropagación. Los hiperparámetros como la tasa de aprendizaje, el número de épocas y el tamaño de los lotes se configuran con el fin de optimizar el rendimiento del modelo, que se entrena y valida iterativamente para garantizar su capacidad de generalización.

In [161]:
# Hiperparámetros
learning_rate = 0.001
batch_size = 128
epochs = 20

# Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

In [162]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [163]:
history = model.fit(train_dataset, 
                    validation_data=validation_dataset, 
                    epochs=epochs)

Epoch 1/20
272/272 [==============================] - 14s 49ms/step - loss: 2.2790 - accuracy: 0.3989 - val_loss: 1.2583 - val_accuracy: 0.6685
Epoch 2/20
272/272 [==============================] - 13s 49ms/step - loss: 0.7888 - accuracy: 0.7828 - val_loss: 0.8737 - val_accuracy: 0.7610
Epoch 3/20
272/272 [==============================] - 13s 48ms/step - loss: 0.4947 - accuracy: 0.8689 - val_loss: 0.7342 - val_accuracy: 0.8141
Epoch 4/20
272/272 [==============================] - 13s 49ms/step - loss: 0.3636 - accuracy: 0.9028 - val_loss: 0.6306 - val_accuracy: 0.8338
Epoch 5/20
272/272 [==============================] - 14s 51ms/step - loss: 0.2831 - accuracy: 0.9261 - val_loss: 0.5676 - val_accuracy: 0.8476
Epoch 6/20
272/272 [==============================] - 13s 47ms/step - loss: 0.2264 - accuracy: 0.9404 - val_loss: 0.6015 - val_accuracy: 0.8438
Epoch 7/20
272/272 [==============================] - 13s 46ms/step - loss: 0.1821 - accuracy: 0.9526 - val_loss: 0.5544 - val_accuracy:

# Evaluacion del modelo

In [164]:
from sklearn.metrics import precision_score, f1_score

y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

precision = precision_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print(f'Precisión: {precision:.2f}')
print(f'F1-Score: {f1:.2f}')



395/395 [==============================] - 1s 1ms/step
Precisión: 0.89
F1-Score: 0.89


In [165]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Hacer predicciones con el modelo
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convertir las predicciones a clases

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Generar el reporte de clasificación
class_report = classification_report(y_test, y_pred_classes)

# Presentar los resultados
print("Matriz de Confusión:")
print(conf_matrix)
print("\nReporte de Clasificación:")
print(class_report)

395/395 [==============================] - 1s 1ms/step
Matriz de Confusión:
[[ 30  23   0 ...   0   0   0]
 [  3 645  60 ...   0   0   0]
 [  0  20 698 ...   0   0   0]
 ...
 [  0   1   0 ...  73   0   0]
 [  0   0   0 ...   0  49   0]
 [  0   0   0 ...   0   1  89]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.71      0.50      0.59        60
           1       0.83      0.90      0.86       720
           2       0.85      0.93      0.89       750
           3       0.87      0.94      0.90       450
           4       0.91      0.87      0.89       660
           5       0.95      0.85      0.90       630
           6       0.98      0.79      0.87       150
           7       0.91      0.85      0.88       450
           8       0.86      0.88      0.87       450
           9       0.91      0.96      0.93       480
          10       0.96      0.97      0.96       660
          11       0.89      0.87      0.88       420
  

## Interpretación

Como se puede observar la precisión es considerabelemente alta para la aplicación dada, reconocimiento de señales de tránsito (sin embargo siempre es necesario evaluar con mayor precisión para justificar esta métrica), esto significa que el 88% de las predicciones del modelo son correctas.
Mientras que F1-Score, con un 0.87, indica que el modelo tiene un buen equilibrio entre precisión y exhaustividad, significando que no solamente es capaz de predecir correctamente los casos positivos, sion que también detecta la mayoria de estos.

El desempeño varía significativamente entre las clases. Por ejemplo, la clase 0 tiene una precisión y F1-score bajos (0.70 y 0.47, respectivamente), indicando que el modelo tiene dificultades para predecir correctamente esta clase. En contraste, clases como la 10 y la 13 tienen una precisión y F1-score muy altos (0.95 y 0.97, respectivamente), lo que sugiere que el modelo predice estas clases con alta precisión y consistencia. Las métricas de recall también varían, con algunas clases como la 6 y la 19 mostrando un recall bajo, lo que indica que el modelo no está capturando todas las instancias de estas clases. 